In [2]:
%reset -f
import sys
sys.path.insert(0, '..')
from wavhandler import Dataset
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from utils_train import train_test_val_split, TrainConfiguration, train_generator, valid_generator,mosquito_data_split, train_model_dl
seed = 42
np.random.seed(seed=seed)

splitting = "random"
data_setting = "raw"
model_setting = "conv1d"

assert splitting in ['random','randomcv','custom'], "Wrong splitting method given."
assert data_setting in ['raw','stft','psd_dB'], "Wrong data settting given."
assert model_setting in ['wavenet','lstm','gru','conv1d','conv1d_psd',
                        'DenseNet121','DenseNet169','DenseNet201',
                        'InceptionResNetV2','VGG16','VGG19'], "Wrong model setting given"

data = Dataset('Wingbeats')
print(data.target_classes)

print(f'SPLITTING DATA {splitting}')
X_train, X_val, X_test, y_train, y_val, y_test = mosquito_data_split(splitting=splitting, dataset=data)

['Ae. aegypti', 'Ae. albopictus', 'An. arabiensis', 'An. gambiae', 'C. pipiens', 'C. quinquefasciatus']
SPLITTING DATA random
Species: Ae. aegypti.
Read 85553 filenames in 1.70 seconds.
Species: Ae. albopictus.
Read 20231 filenames in 1.71 seconds.
Species: An. arabiensis.
Read 19297 filenames in 1.66 seconds.
Species: An. gambiae.
Read 49471 filenames in 1.66 seconds.
Species: C. pipiens.
Read 30415 filenames in 1.69 seconds.
Species: C. quinquefasciatus.
Read 74599 filenames in 1.68 seconds.


In [3]:
from wavhandler import make_df_parallel
x_train = make_df_parallel(names=X_train, setting='raw')
x_test = make_df_parallel(names=X_test, setting='raw')

In [4]:
x_train = np.expand_dims(x_train.values, axis=-1)
x_test = np.expand_dims(x_test.values, axis=-1)

In [5]:
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D
from tensorflow.keras.models import Model

input_ts = Input(shape=(5000,1))

x = Conv1D(32, 3, activation="relu", padding="same")(input_ts)
x = MaxPooling1D(2 )(x)

x = Conv1D(1, 3, activation="relu", padding="same")(x)

encoded = MaxPooling1D(2 )(x)

encoder = Model(input_ts, encoded)

x = Conv1D(16, 3, activation="relu", padding="same")(encoded)
x = UpSampling1D(2)(x) 

x = Conv1D(32, 3, activation='relu', padding="same")(x)
x = UpSampling1D(2)(x)

decoded = Conv1D(1, 1, activation='tanh', padding='same')(x)

convolutional_autoencoder = Model(input_ts, decoded)

convolutional_autoencoder.summary()

optimizer = "nadam"
loss = "mean_absolute_error"

convolutional_autoencoder.compile(optimizer=optimizer, loss=loss)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5000, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 5000, 32)          128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 2500, 32)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2500, 1)           97        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1250, 1)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1250, 16)          64        
_________________________________________________________________
up_sampling1d (UpSampling1D) (None, 2500, 16)          0   

In [6]:
convolutional_autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=32,
                shuffle=True,
                verbose = 1)

Train on 61439 samples
Epoch 1/10
61439/61439 [==============================] - 64s 1ms/sample - loss: 0.0027
Epoch 2/10
61439/61439 [==============================] - 54s 876us/sample - loss: 0.0023
Epoch 3/10
61439/61439 [==============================] - 53s 865us/sample - loss: 0.0022
Epoch 4/10
61439/61439 [==============================] - 54s 880us/sample - loss: 0.0021
Epoch 5/10
61439/61439 [==============================] - 55s 892us/sample - loss: 0.0021
Epoch 6/10
61439/61439 [==============================] - 54s 883us/sample - loss: 0.0021
Epoch 7/10
61439/61439 [==============================] - 53s 867us/sample - loss: 0.0021
Epoch 8/10
61439/61439 [==============================] - 55s 901us/sample - loss: 0.0021
Epoch 9/10
61439/61439 [==============================] - 55s 895us/sample - loss: 0.0021
Epoch 10/10
61439/61439 [==============================] - 55s 897us/sample - loss: 0.0021
